In [1]:
import torch
import numpy as np
from einops import einsum

from diffusers import DiffusionPipeline
from evotorch import Problem
from evotorch.decorators import vectorized
from evotorch.algorithms import CMAES, SNES, CEM
from evotorch.logging import StdOutLogger, PandasLogger
from noise_injection_pipelines.sampling_pipelines import (
    SDXLSamplingPipeline,
    SD3SamplingPipeline,
)
from fitness.fitness_fn import (
    brightness,
    clip_fitness_fn,
    compose_fitness_fns,
    relative_luminance,
    Novelty,
    pickscore_fitness_fn,
    aesthetic_fitness_fn,
    imagereward_fitness_fn,
)
from evo.vectorized_problem import VectorizedProblem
from diffusers.utils import pt_to_pil, numpy_to_pil
import matplotlib.pyplot as plt

from noise_injection_pipelines.noise_injection import (
    rotational_transform,
    noise
)
from noise_injection_pipelines.initialization import randn_intialization, rotation_initalization
from functools import partial

/opt/homebrew/Caskroom/mambaforge/base/envs/diffusion-traj/lib/python3.12/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


'NoneType' object has no attribute 'cadam32bit_grad_fp32'
Imagereward not able to be imported, see https://github.com/THUDM/ImageReward/tree/main for install


In [2]:
model_name = "stabilityai/sdxl-turbo"
model_name = "stable-diffusion-v1-5/stable-diffusion-v1-5"
pipe = DiffusionPipeline.from_pretrained(
    model_name, torch_dtype=torch.float16, use_safetensors=True, variant="fp16"
)
# pipe.enable_xformers_memory_efficient_attention()
# pipe.enable_model_cpu_offload()

model_index.json:   0%|          | 0.00/541 [00:00<?, ?B/s]

Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

(…)ature_extractor/preprocessor_config.json:   0%|          | 0.00/342 [00:00<?, ?B/s]

text_encoder/config.json:   0%|          | 0.00/617 [00:00<?, ?B/s]

safety_checker/config.json:   0%|          | 0.00/4.72k [00:00<?, ?B/s]

...

vae/config.json:   0%|          | 0.00/547 [00:00<?, ?B/s]

diffusion_pytorch_model.fp16.safetensors:   0%|          | 0.00/1.72G [00:00<?, ?B/s]

model.fp16.safetensors:   0%|          | 0.00/608M [00:00<?, ?B/s]

tokenizer/tokenizer_config.json:   0%|          | 0.00/806 [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

In [3]:
problem_split = 6
num_inference_steps = 15
noise_scale = 1.0
prompt = "A beautiful landscape painting"
sample_fn = SDXLSamplingPipeline(
    pipe,
    prompt=prompt,
    num_inference_steps=num_inference_steps,
    generator=torch.Generator().manual_seed(0),
    guidance_scale=5.0
)

# image_rew = imagereward_fitness_fn([prompt], device=pipe.device, dtype=pipe.dtype)
fit = compose_fitness_fns([brightness], [1])

ValueError: not enough values to unpack (expected 4, got 2)

In [6]:
mean_scale = 0.01
initial_bounds = (-0.1, 0.1)
injection_steps = num_inference_steps
sample_fn.inject_multiple_noise_scale = noise_scale

fitness_fn, inner_fn, centroid, solution_length = rotational_transform(
    sample_fn,
    fit,
    sample_fn.latents.shape,
    center=sample_fn.latents,
    dtype=pipe.dtype,
)

problem = VectorizedProblem(
    "max",
    fitness_fn,
    solution_length=solution_length,
    initial_bounds=initial_bounds,
    dtype=np.dtype("float32"),
    splits=problem_split,
    # initialization=None,
    initialization=partial(rotation_initalization, solution_length=solution_length, latents_shape=sample_fn.latents.shape, stdev=0.1),
)
# searcher = CMAES(problem, stdev_init=1, separable=True, csa_squared=True)
searcher = SNES(problem, stdev_init=5)
logger = StdOutLogger(searcher)
pandas_logger = PandasLogger(searcher)
# print(f"pop. size: {searcher.popsize}")

[2025-02-08 13:33:21] INFO     <36198> evotorch.core: Instance of `VectorizedProblem` (id:6416522400) -- The `dtype` for the problem's decision variables is set as torch.float32
[2025-02-08 13:33:21] INFO     <36198> evotorch.core: Instance of `VectorizedProblem` (id:6416522400) -- `eval_dtype` (the dtype of the fitnesses and evaluation data) is set as torch.float32
[2025-02-08 13:33:21] INFO     <36198> evotorch.core: Instance of `VectorizedProblem` (id:6416522400) -- The `device` of the problem is set as cpu
[2025-02-08 13:33:21] INFO     <36198> evotorch.core: Instance of `VectorizedProblem` (id:6416522400) -- The number of actors that will be allocated for parallelized evaluation is 0


In [7]:
a = sample_fn()
plt.imshow(numpy_to_pil(a)[0])
plt.show()
print(fit(a[0]))
with torch.no_grad():
    for step in range(200):
        searcher.step()
        best_idx = searcher.population.argbest()
        x = searcher.population[best_idx].values

        a = inner_fn(x)
        plt.imshow(numpy_to_pil(a)[0])
        plt.show()

  0%|          | 0/15 [00:00<?, ?it/s]

KeyboardInterrupt: 

## SDXL Quantized

In [ ]:
from diffusers import BitsAndBytesConfig as DiffusersBitsAndBytesConfig
from diffusers import UNet2DConditionModel

In [ ]:
model_name = "stabilityai/sdxl-turbo"

quant_config = DiffusersBitsAndBytesConfig(
    load_in_4bit=True, bnb_4bit_compute_dtype=torch.float16
)

unet = UNet2DConditionModel.from_pretrained(
    model_name,
    subfolder="unet",
    quantization_config=quant_config,
    torch_dtype=torch.float16,
)

pipe = DiffusionPipeline.from_pretrained(
    model_name, torch_dtype=torch.float16, use_safetensors=True, unet=unet
)
# pipe.enable_xformers_memory_efficient_attention()
# pipe.enable_model_cpu_offload()

In [ ]:
problem_split = 6
num_inference_steps = 15
noise_scale = 1.0
prompt = "A small domesticated carnivorous mammal with soft fur, a short snout, and retractable claws. It is widely kept as a pet or for catching mice, and many breeds have been developed."
sample_fn = SDXLSamplingPipeline(
    pipe,
    prompt=prompt,
    num_inference_steps=num_inference_steps,
    generator=torch.Generator(device=pipe.device).manual_seed(0),
    guidance_scale=5.0
)

image_rew = imagereward_fitness_fn([prompt], device=pipe.device, dtype=pipe.dtype)
fit = compose_fitness_fns([image_rew], [1])

In [ ]:
mean_scale = 0.01
initial_bounds = (-0.1, 0.1)
injection_steps = num_inference_steps
sample_fn.inject_multiple_noise_scale = noise_scale

fitness_fn, inner_fn, centroid, solution_length = rotational_transform(
    sample_fn,
    fit,
    sample_fn.latents.shape,
    device=pipe.device,
    center=sample_fn.latents,
    dtype=pipe.dtype,
)

problem = VectorizedProblem(
    "max",
    fitness_fn,
    solution_length=solution_length,
    initial_bounds=initial_bounds,
    dtype=np.dtype("float32"),
    splits=problem_split,
    # initialization=None,
    initialization=partial(rotation_initalization, solution_length=solution_length, latents_shape=sample_fn.latents.shape, stdev=0.0),
)
# searcher = CMAES(problem, stdev_init=1, separable=True, csa_squared=True)
searcher = SNES(problem, stdev_init=0.1)
logger = StdOutLogger(searcher)
pandas_logger = PandasLogger(searcher)
# print(f"pop. size: {searcher.popsize}")

In [ ]:
a = sample_fn()
plt.imshow(numpy_to_pil(a)[0])
plt.show()
print(fit(a[0]))
with torch.no_grad():
    for step in range(200):
        searcher.step()
        best_idx = searcher.population.argbest()
        x = searcher.population[best_idx].values

        a = inner_fn(x)
        plt.imshow(numpy_to_pil(a)[0])
        plt.show()

# SD3

In [ ]:
from diffusers import BitsAndBytesConfig, SD3Transformer2DModel
from diffusers import StableDiffusion3Pipeline
import torch

# model_id = "stabilityai/stable-diffusion-3.5-medium"
model_id = "tensorart/stable-diffusion-3.5-medium-turbo"

nf4_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)
model_nf4 = SD3Transformer2DModel.from_pretrained(
    model_id,
    subfolder="transformer",
    quantization_config=nf4_config,
    torch_dtype=torch.float16
)

pipeline = StableDiffusion3Pipeline.from_pretrained(
    model_id, 
    transformer=model_nf4,
    torch_dtype=torch.float16
)


pipeline.enable_xformers_memory_efficient_attention()
pipeline.enable_model_cpu_offload()

In [ ]:
prompt = "A whimsical and creative image depicting a hybrid creature that is a mix of a waffle and a hippopotamus, basking in a river of melted butter amidst a breakfast-themed landscape. It features the distinctive, bulky body shape of a hippo. However, instead of the usual grey skin, the creature's body resembles a golden-brown, crispy waffle fresh off the griddle. The skin is textured with the familiar grid pattern of a waffle, each square filled with a glistening sheen of syrup. The environment combines the natural habitat of a hippo with elements of a breakfast table setting, a river of warm, melted butter, with oversized utensils or plates peeking out from the lush, pancake-like foliage in the background, a towering pepper mill standing in for a tree.  As the sun rises in this fantastical world, it casts a warm, buttery glow over the scene. The creature, content in its butter river, lets out a yawn. Nearby, a flock of birds take flight"
prompt = "A small domesticated carnivorous mammal with soft fur, a short snout, and retractable claws. It is widely kept as a pet or for catching mice, and many breeds have been developed."

sample_fn = SD3SamplingPipeline(
    pipeline,
    prompt=prompt,
    num_inference_steps=15,
    generator=torch.Generator(device=pipeline.device).manual_seed(1),
    guidance_scale=4.5,
    add_noise=False
)

In [ ]:
image_rew = imagereward_fitness_fn(
    [prompt], device=pipeline.device, dtype=pipeline.dtype
)
fit = compose_fitness_fns([image_rew], [1])

mean_scale = 0.000
initial_bounds = (-1, 1)
problem_split = 4

# fitness_fn, inner_fn, centroid, solution_length = rotational_transform(
#     sample_fn,
#     fit,
#     sample_fn.latents.shape,
#     device=pipeline.device,
#     center=sample_fn.latents,
#     dtype=pipeline.dtype,
# )

fitness_fn, inner_fn, centroid, solution_length = noise(
    sample_fn,
    fit,
    sample_fn.latents.shape,
    device=pipeline.device,
    dtype=pipeline.dtype,
)


problem = VectorizedProblem(
    "max",
    fitness_fn,
    solution_length=solution_length,
    initial_bounds=initial_bounds,
    dtype=np.dtype("float32"),
    splits=problem_split,
    # initialization=partial(rotation_initalization, solution_length=solution_length, latents_shape=sample_fn.latents.shape, stdev=0.00),
    initialization=partial(randn_intialization, stdev=1),
    device=pipeline.device,
)
searcher = CMAES(
    problem,
    stdev_init=0.5,
    separable=True,
    center_init=sample_fn.latents.flatten(),
    # center_init=torch.zeros_like(sample_fn.latents.flatten()),
    # csa_squared=True,
)
searcher = SNES(problem, stdev_init=0.1, center_init=sample_fn.latents.flatten())
logger = StdOutLogger(searcher)
pandas_logger = PandasLogger(searcher)

In [ ]:
a = sample_fn()
plt.imshow(numpy_to_pil(a)[0])
plt.show()
print(fit(a[0]))
with torch.no_grad():
    for step in range(50):
        searcher.step()
        best_idx = searcher.population.argbest()
        x = searcher.population[best_idx].values

        a = inner_fn(x)
        plt.imshow(numpy_to_pil(a)[0])
        plt.show()

In [4]:
from evotorch import Problem
from evotorch.algorithms import SNES
from evotorch.logging import StdOutLogger
import torch


# Define a function to minimize
def sphere(x: torch.Tensor) -> torch.Tensor:
    return torch.sum(x.pow(2.0))


# Define a Problem instance wrapping the function
# Solutions have length 10
problem = Problem("min", sphere, solution_length=10, initial_bounds=(-1, 1))

# Instantiate a searcher
searcher = SNES(problem, stdev_init=5)

# Create a logger
logger = StdOutLogger(searcher)

# Evolve!
searcher.run(3)

[2025-02-08 13:39:36] INFO     <36321> evotorch.core: Instance of `Problem` (id:5804718992) -- The `dtype` for the problem's decision variables is set as torch.float32
[2025-02-08 13:39:36] INFO     <36321> evotorch.core: Instance of `Problem` (id:5804718992) -- `eval_dtype` (the dtype of the fitnesses and evaluation data) is set as torch.float32
[2025-02-08 13:39:36] INFO     <36321> evotorch.core: Instance of `Problem` (id:5804718992) -- The `device` of the problem is set as cpu
[2025-02-08 13:39:36] INFO     <36321> evotorch.core: Instance of `Problem` (id:5804718992) -- The number of actors that will be allocated for parallelized evaluation is 0
         iter : 1
    mean_eval : 261.0331115722656
  median_eval : 238.25753784179688
pop_best_eval : 74.1915054321289
    best_eval : 74.1915054321289
   worst_eval : 442.37646484375

         iter : 2
    mean_eval : 195.61996459960938
  median_eval : 213.41314697265625
pop_best_eval : 54.15727996826172
    best_eval : 54.15727996826172


In [7]:
searcher.status

<LazyStatusDict
    center = tensor([-1.6826, -1.9729, -0.2225, -1.8703, -0.1467, -0.4051, -1.2375, -1.7636,
         0.8631,  1.4204])
    stdev = tensor([4.3781, 4.5753, 4.6805, 4.0276, 4.4291, 4.1138, 4.7932, 4.0510, 4.5696,
        4.7234])
    mean_eval = 171.62869262695312
    median_eval = 125.94084167480469
    pop_best_eval = 89.4813003540039
    pop_best = <Solution values=tensor([-3.3027, -2.9052, -3.7853,  3.7399, -0.9998,  0.9296, -1.7739,  0.8803,
         1.3691,  5.8445]), evals=tensor([89.4813])>
    iter = 3
    best = <Solution values=tensor([ 2.4455e+00, -2.1970e+00, -8.0808e-01, -1.3402e+00, -2.6239e+00,
        -1.1752e+00, -3.0875e+00,  1.2870e-03,  1.4489e+00,  4.5829e+00]), evals=tensor([54.1573])>
    worst = <Solution values=tensor([-10.9263,   5.7003,  -1.8264,   7.9866,  -3.0197,  -3.4721,   2.6687,
         -1.0535, -12.8548,   5.3595]), evals=tensor([442.3765])>
    best_eval = 54.15727996826172
    worst_eval = 442.37646484375
>